In [1]:
import pandas as pd
import sqlite3

In [25]:
datos_properati = pd.read_csv("datasets_inmuebles/df_compra_caba.csv")
datos_properati.head(2)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Fecha
0,422c5a4888286cf5966e548135c0c524da40696c,2014-11-22,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,"-34.622636073,-58.391393158",...,NaN,21835.294118,NaN,1.0,NaN,http://www.properati.com.ar/jtaf_venta_departa...,Corredor Responsable: Gustavo Guastello - C.U....,"MONOAMBIENTE EN 9° PISO ""B"" FTE. ENTREGA ESIMA...",https://thumbs4.properati.com/1/4GtRbgOAz3Mvf7...,2015-01-01
1,79ad5a6361e58cbb076e2c00f3169a606b06240a,2014-12-02,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,"-34.601281,-58.379588",...,1595.238095,1970.588235,NaN,2.0,570.0,http://www.properati.com.ar/jx0u_venta_departa...,Departamento APTO PROFESIONAL de dos ambientes...,Departamento venta,https://thumbs4.properati.com/9/S2GZ9iMdngVb7Z...,2015-01-01


In [26]:
concat_properati_ml = pd.read_csv("datasets_inmuebles/concat_properati_ml.csv")
concat_properati_ml.head(2)

,Fecha,Precio
0,2015-01,2214
1,2015-02,2258


In [45]:
stocks = pd.read_csv("datasets_yahoofinance/concatenado/empresas_grandes.csv")
stocks.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,empresa,industria,market_cap,ticker
0,2018-01-02,52.330002,53.099998,51.900002,52.990002,51.647568,4084700,American Airlines Group Inc,Airlines,11712362496,AAL
1,2018-01-03,52.860001,52.860001,52.060001,52.340000,51.014023,5074900,American Airlines Group Inc,Airlines,11712362496,AAL


In [46]:
stocks.rename(columns={'Date':'Fecha','Adj Close': 'Adj_Close'}, inplace=True)

In [29]:
db_path = "database/tp_final_db.db"

conn = sqlite3.connect(db_path)

In [30]:
stocks.to_sql(name='stocks', con=conn, if_exists="replace", index = False)

concat_properati_ml.to_sql(name='concat_properati_ml', con=conn, if_exists="replace", index = False)

datos_properati.to_sql(name='properati', con=conn, if_exists="replace", index = False)

# Close the connection
conn.close()

# Enviando las tablas a BQ

In [33]:
from google.oauth2 import service_account # para generar conexion con BigQuery
#bq_cred = service_account.Credentials.from_service_account_file('./tokens/gcp_token2.json')
bq_cred = service_account.Credentials.from_service_account_file('./tokens/gcp_token_tp_final.json')

bq_cred

In [37]:
import pandas_gbq # para interactuar con BQ

datos_properati.to_gbq(destination_table= 'tp_final.properati', project_id= 'tp-final-big-data',
                   if_exists= 'replace', credentials = bq_cred)

In [43]:
concat_properati_ml.to_gbq(destination_table= 'tp_final.concat_properati_ml', project_id= 'tp-final-big-data',
                   if_exists= 'replace', credentials = bq_cred)

TypeError: to_gbq() got an unexpected keyword argument 'reorder_columns'

In [47]:
stocks.to_gbq(destination_table= 'tp_final.stocks', project_id= 'tp-final-big-data',
                   if_exists= 'replace', credentials = bq_cred)

In [41]:
concat_properati_ml

,Fecha,Precio
0,2015-01,2214
1,2015-02,2258
2,2015-03,2265
3,2015-04,2299
4,2015-05,2381
...,...,...
93,2022-10,2288
94,2022-11,2289
95,2022-12,2268
96,2023-03,2295
